# The 15 puzzle
## Using Dynamic Programming

In this notebook, we will try to develop a dp algorithm to solve the 15 puzzle
The puzzle basically consists of a 4x4 box with 1x1 tiles numbered from 1 to 15. There is a single tile vacancy in the box, which can be used to shift tiles around.

Our goal is to reach the solved state :

<table>
  <tr>
    <td style="border: 1px solid black; text-align: center; width: 50px;">1</td>
    <td style="border: 1px solid black; text-align: center; width: 50px;">2</td>
    <td style="border: 1px solid black; text-align: center; width: 50px;">3</td>
    <td style="border: 1px solid black; text-align: center; width: 50px;">4</td>
  </tr>
  <tr>
    <td style="border: 1px solid black; text-align: center;">5</td>
    <td style="border: 1px solid black; text-align: center;">6</td>
    <td style="border: 1px solid black; text-align: center;">7</td>
    <td style="border: 1px solid black; text-align: center;">8</td>
  </tr>
  <tr>
    <td style="border: 1px solid black; text-align: center;">9</td>
    <td style="border: 1px solid black; text-align: center;">10</td>
    <td style="border: 1px solid black; text-align: center;">11</td>
    <td style="border: 1px solid black; text-align: center;">12</td>
  </tr>
  <tr>
    <td style="border: 1px solid black; text-align: center;">13</td>
    <td style="border: 1px solid black; text-align: center;">14</td>
    <td style="border: 1px solid black; text-align: center;">15</td>
    <td style="border: 1px solid black; text-align: center;"> </td>
  </tr>
</table>



Firstly, we need to implement the structure of the puzzle i.e. we need to initialise the boarrd, locate the blank square, find possible actions, find the next state for every action, check if the board is solved and create a unique hash for the state.

It is important to note that not every permutation can reach the goal, hence we must generate possible permutations from the goal state itself.

This will be our strategy:
- For a given puzzle(via scrambler)
- Using a copy of the puzzle, mask everything but the first row(phase1)
- Use precomputed policy for phase 1, apply it to the masked state and store actions in an array
- return the actions and use them to solve the original puzzle
- mask everything but the second row and continue similarly

To compute the policy, we will use policy iteration. As explained in the medium page, the set of states is very large(~$ \frac{16!}{2} $). 

So we will use masking and generate all possible states(every state need not be relevant to our puzzle, as many permutations are unsolvable) using distinct_permutations in more_itertools

Lets begin:

In [86]:
import numpy as np
import random
from more_itertools import distinct_permutations
import time

In [87]:
set_of_acts=["Up","Down","Left","Right"]  #set of actions possible
gamma=0.9 
num_cells=16

first_row_solved = [1, 2, 3, 4, num_cells, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
second_row_solved= [-1, -1, -1, -1, 5, 6, 7, 8, num_cells, 0, 0, 0, 0, 0, 0, 0]
all_rows_solved= [-1, -1, -1, -1, -1, -1, -1, -1, 9, 10, 11, 12, 13, 14, 15, num_cells]

def partial_solve(state):     #checks if a state is partially solved
    if (state[:4] == first_row_solved[:4]):
        return True
    if (state[:8] == second_row_solved[:8]):
        return True
    if (state == all_rows_solved):
        return True
    return False


def mask(array, row):
    arr = array.copy()
    array = []  #fresh array to build the new masked array
    
    if row == 1:
        #Mask all elements except for 1, 2, 3, 4, and num_cells
        for x in arr:
            if x not in [1, 2, 3, 4, num_cells]:
                array.append(0)  # Mask by setting other values to 0
            else:
                array.append(x)
                
    elif row == 2:
        #Mask the first row elements and others except 5, 6, 7, 8, and num_cells
        for x in arr:
            if x in [1, 2, 3, 4]:
                array.append(-1)  #Mask the first row by appending -1
            elif x not in [5, 6, 7, 8, num_cells]:
                array.append(0)  #Mask other values by appending 0
            else:
                array.append(x)

    elif row == 3:
        for x in arr:
            if x in [1, 2, 3, 4, 5, 6, 7, 8]:
                array.append(-1)
            else:
                array.append(x)
    return array


def actions(arr):
    actions = []  # based on location of empty cell, find actions
    empty_cell_index = arr.index(num_cells)
    row = empty_cell_index // 4
    col = empty_cell_index % 4
    if row > 0:
        actions.append("Up")
    if row < 3:
        actions.append("Down")
    if col > 0:
        actions.append("Left")
    if col < 3:
        actions.append("Right")
    return actions

def mask_move(state, action):    #this function will be used to move masked states without moving solved rows(-1's)
    i = state.index(num_cells) 
    row = i // 4
    col = i % 4
    
    if ((action == "Up") and (row > 0) and (state[i - 4] != -1)):  
        state[i], state[i - 4] = state[i - 4], state[i]  

    elif ((action == "Down") and (row < 3) and (state[i + 4] != -1)):  
        state[i], state[i + 4] = state[i + 4], state[i]  

    elif ((action == "Left") and (col > 0) and (state[i - 1] != -1)):  
        state[i], state[i - 1] = state[i - 1], state[i]  

    elif ((action == "Right") and (col < 3) and (state[i + 1] != -1)):  
        state[i], state[i + 1] = state[i + 1], state[i]  

    return state  

def move(state, action):  #this is a normal move function used to move the actual puzzle
    i = state.index(num_cells)
    
    if ((action == "Up")):  
        state[i], state[i - 4] = state[i - 4], state[i]  

    elif ((action == "Down")):  
        state[i], state[i + 4] = state[i + 4], state[i]  

    elif ((action == "Left")):  
        state[i], state[i - 1] = state[i - 1], state[i]  

    elif ((action == "Right")): 
        state[i], state[i + 1] = state[i + 1], state[i]  
    
    return state 


def reward(arr):
    if partial_solve(arr):
        return 1000       #huge reward for solving a row 
    return -1

def copy(state):
    return state[:]

def generate_states(distinct_numbers, num_minus_ones, num_zeros):
    
    arr = distinct_numbers + [0] * num_zeros
    result=[]
    #generate distinct permutations of zeros and distinct numbers
    for perm in distinct_permutations(arr):
        result.append([-1] * num_minus_ones + list(perm))   
    return result




In [88]:
def policy_improvement(V, Policy, states):
    stable = True                #standard policy improvement like jack car rental
    for state in states:
        state_tuple = tuple(state)  #converting state to tuple for dict use    (tuple OBjeCT iS nOT mUtAbLE T_T)
        prev_action = Policy[state_tuple] 
        if partial_solve(state): 
            continue
        
        max_value = -np.inf  
        best_action = ''
    
        for action in actions(state):
            new_state = copy(state)
            new_state = mask_move(new_state, action)  #perform every action
            action_val = reward(new_state) + gamma*V[tuple(new_state)]  #get the reward
            
            if action_val > max_value:  #get best action
                max_value = action_val
                best_action = action
        
        Policy[state_tuple] = best_action  #update policy for a given value function
        if best_action != prev_action:
            stable = False
    
    return stable

def policy_evaluation(V, Policy, states):
    for state in states:
        state_tuple = tuple(state)  
        if partial_solve(state):  #every policy will contain actions only till its partial-goal state, hence skip goal state
            continue
    
        action = Policy[state_tuple]  
        new_state = copy(state)
        new_state = mask_move(new_state, action)  
        V[state_tuple] = reward(new_state) + gamma*V[tuple(new_state)] #for a given policy update the value function

def policy_iteration(states):
    V = {tuple(state): 0 for state in states}  
    Policy = {tuple(state): '' for state in states} 

    stable = False
    itr=0
    while not stable:
        itr+=1
        stable = policy_improvement(V, Policy, states)  
        policy_evaluation(V, Policy, states)  
    print(itr)   #to check if policy is computing properly(i am dead)
    return Policy  


In [89]:

phase1_states=generate_states([1,2,3,4,16], 0, 11)  #generate all(and extra) phase 1 states
Pol1=policy_iteration(phase1_states)                #get the policy for phase1_states

phase2_states=generate_states([5,6,7,8,16], 4, 7)
Pol2=policy_iteration(phase2_states)

phase3_states=generate_states([9,10,11,12,13,14,15,16], 8, 0)
Pol3=policy_iteration(phase3_states)


36
35
37


In [90]:
step = 0  

def display(puzzle):
    global step  
    
    for i in range(0, 16, 4):
        row = ["|"] 
        for num in puzzle[i:i+4]:  
            row.append(f" {num:2d} ")  
        row.append("|") 
        print("".join(row))  
    step += 1
    print(f"Step: {step}")
    print("----------------------------")

In [91]:
def solve(puzzle):
    state = puzzle.copy()

    def process_phase(masked_state, policy, phase):
        actions = []
        while True:
            action = ''
            if tuple(masked_state) in policy:
                action = policy[tuple(masked_state)]
                actions.append(action)
            if action not in set_of_acts:
                break 
            masked_state = move(masked_state, action)
        return actions

    # Phase 1
    print("Starting phase 1")
    time.sleep(1.5)
    masked_state = mask(state, 1)
    actions_phase1 = process_phase(masked_state, Pol1, 1)

    for action in actions_phase1:
        state = move(state, action)
        display(state)
        time.sleep(1)

    # Phase 2   
    print("Starting phase 2")
    time.sleep(1.5)
    masked_state = mask(state, 2)
    actions_phase2 = process_phase(masked_state, Pol2, 2)

    for action in actions_phase2:
        state = move(state, action)
        display(state)
        time.sleep(1)

    # Phase 3
    print("Starting phase 3")
    time.sleep(1.5)
    masked_state = mask(state, 3)
    actions_phase3 = process_phase(masked_state, Pol3, 3)

    for action in actions_phase3:
        state = move(state, action)
        display(state)
        time.sleep(1)
    
    print("SOLVED💃💃💃")
    display(state)
step=0
# YAYYY, we are done with the main code

In [92]:
#puzzle scrambler to create a random puzzle

def scramble_puzzle(num_steps=300):
    solved_puzzle = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]  
    current_state = solved_puzzle.copy()
    for _ in range(num_steps):
        act=actions(current_state)
        action = random.choice(act)  # Choose a random move
        current_state = move(current_state, action)  # Apply the move
    return current_state

In [93]:
#repeat this block of code for more puzzle solves
puzzle = scramble_puzzle()
print("Let us solve the following puzzle:-")
step=0
display(puzzle)
time.sleep(1.5)
solve(puzzle)

Let us solve the following puzzle:-
|  5   2   8   4 |
| 10  16   1   9 |
| 15  11   7  12 |
| 13  14   3   6 |
Step: 1
----------------------------
Starting phase 1
|  5   2   8   4 |
| 10  11   1   9 |
| 15  16   7  12 |
| 13  14   3   6 |
Step: 2
----------------------------
|  5   2   8   4 |
| 10  11   1   9 |
| 15   7  16  12 |
| 13  14   3   6 |
Step: 3
----------------------------
|  5   2   8   4 |
| 10  11   1   9 |
| 15   7   3  12 |
| 13  14  16   6 |
Step: 4
----------------------------
|  5   2   8   4 |
| 10  11   1   9 |
| 15   7   3  12 |
| 13  16  14   6 |
Step: 5
----------------------------
|  5   2   8   4 |
| 10  11   1   9 |
| 15  16   3  12 |
| 13   7  14   6 |
Step: 6
----------------------------
|  5   2   8   4 |
| 10  16   1   9 |
| 15  11   3  12 |
| 13   7  14   6 |
Step: 7
----------------------------
|  5   2   8   4 |
| 10   1  16   9 |
| 15  11   3  12 |
| 13   7  14   6 |
Step: 8
----------------------------
|  5   2  16   4 |
| 10   1   8   9 |
| 15 